In [67]:
import pandas as pd
import numpy as np
import math

pd.options.plotting.backend = "plotly"
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from pint import UnitRegistry

ureg = UnitRegistry()
Q_ = ureg.Quantity

In [68]:
resid_5_link = {'resid': r'https://raw.githubusercontent.com/jtarriela/FDA_Blood_Pump/main/Core/SBES_15M_BP/1.5e-5/Residuals/resid_all.out'}
# consolidated all data in python script
#uploaded the results to repo
# called clean results here
residual_df = pd.read_csv(resid_5_link['resid'], header=0, sep=" ")
resid_col_list = residual_df.columns

In [69]:
rev_complete = 5
case = {'rpm': Q_(2500, 'rpm'),
        'blades': 4,
        'timestep': Q_(2.5E-5, 'sec')}

rps = case['rpm'].to('rps')  #revolutions per second
sec_per_rev = 1 / rps * Q_(1, 'revolutions')
sec_per_rev = sec_per_rev.to('sec')
sec_per_blade_pass = sec_per_rev / case['blades']
step_per_blade_pass = sec_per_blade_pass / case['timestep']
step_per_rev = step_per_blade_pass * 4

### Change to suit bin size--
### Either per blade pass or per revolution for smoothing
bin_size = int((step_per_rev * rev_complete) / step_per_blade_pass)

print(f'For Specified Timestep: {case["timestep"]} \n'
      f'Seconds per revolution: {sec_per_rev} \n'
      f'Seconds per blade pass: {sec_per_blade_pass} \n'
      f'Time steps per blade pass: {step_per_blade_pass} \n'
      f'Time steps per rotation: {step_per_blade_pass * 4}\n\n'
      f'One bin per blade pass\n\n'
      f'Number of bins: {bin_size}\n\n')

bin_df = residual_df.copy()
# Use df.copy(): indexing a DataFrame returns a reference to the initial DataFrame.
# Thus, changing the subset will change the initial DataFrame.
# Thus, you'd want to use the copy if you want to make sure the initial DataFrame shouldn't change.

col_list = bin_df.columns[0:].copy()

bin_df["Bins"] = pd.cut(bin_df[col_list[0]], bins=bin_size)
bin_df["bin_centres"] = bin_df["Bins"].apply(lambda x: x.mid)
bin_df['Counts'] = bin_df.groupby(['Bins'])[col_list[0]].transform('count')
bin_df['Bin Variance'] = bin_df.groupby(['Bins'])[col_list[0]].transform('var')
for variable in enumerate(resid_col_list):
    # print(variable)
    if variable[0] == 0:
        # skip first column "Time_step"
        pass
    else:
        # print(variable)
        col_bin_var_mean = variable[1] + "_bin_mean"
        col_bin_std = variable[1] + "_std"
        col_bin_sem = variable[1] + "_sem"
        bin_df[col_bin_var_mean] = bin_df.groupby('Bins')[col_list[variable[0]]].transform('mean')
        bin_df[col_bin_std] = bin_df.groupby('Bins')[col_list[variable[0]]].transform('std')
        bin_df[col_bin_sem] = bin_df.groupby('Bins')[col_list[variable[0]]].transform('sem')

bin_df = bin_df.drop_duplicates(subset="Bins")
bin_df = bin_df.reset_index()
bin_df = bin_df.drop(resid_col_list, axis=1)

For Specified Timestep: 2.5e-05 second 
Seconds per revolution: 0.024 second 
Seconds per blade pass: 0.006 second 
Time steps per blade pass: 240.0 dimensionless 
Time steps per rotation: 960.0 dimensionless

One bin per blade pass

Number of bins: 20


For Specified Timestep: 2.5e-05 second 
Seconds per revolution: 0.024 second 
Seconds per blade pass: 0.006 second 
Time steps per blade pass: 240.0 dimensionless 
Time steps per rotation: 960.0 dimensionless

One bin per blade pass

Number of bins: 20




In [70]:
color10_16 = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF',
              '#FECB52']
scatter_opacity = 0.7
mean_opacity = scatter_opacity

fig = make_subplots(
    rows=5,
    cols=1,
    shared_xaxes=False,
    x_title="Time Step",
    y_title="Damage Index")
# subplot_titles=("Point A", "Quadrant 2"))

fig.add_trace(go.Scatter(x=residual_df['Time_Step'],
                         y=residual_df['point-gw-a'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='GW-A',
                         line=dict(color=color10_16[0])),
              row=1, col=1)
fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
                         y=bin_df['point-gw-a_bin_mean'],
                         mode='markers+lines',
                         name='Mean +/- STD',
                         line=dict(color='black'),
                         opacity=.55,
                         error_x_array=bin_df['point-gw-a_std'],
                         showlegend=False),
              row=1, col=1)

fig.add_trace(go.Scatter(x=residual_df['Time_Step'],
                         y=residual_df['point-gw-b'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='GW-B',
                         line=dict(color=color10_16[1])),
              row=2, col=1)
fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
                         y=bin_df['point-gw-b_bin_mean'],
                         mode='markers+lines',
                         name='Mean +/- STD',
                         line=dict(color='black'),
                         opacity=.55,
                         error_x_array=bin_df['point-gw-b_std'],
                         showlegend=False),
              row=2, col=1)

fig.add_trace(go.Scatter(x=residual_df['Time_Step'],
                         y=residual_df['point-gw-c'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='GW-C',
                         line=dict(color=color10_16[2]),
                         showlegend=True),
              row=3, col=1)
fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
                         y=bin_df['point-gw-c_bin_mean'],
                         mode='markers+lines',
                         name='Mean +/- STD',
                         line=dict(color='black'),
                         opacity=.55,
                         error_x_array=bin_df['point-gw-c_std'],
                         showlegend=False),
              row=3, col=1)

fig.add_trace(go.Scatter(x=residual_df['Time_Step'],
                         y=residual_df['point-gw-d'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='GW-D',
                         line=dict(color=color10_16[3]),
                         showlegend=True),
              row=4, col=1)
fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
                         y=bin_df['point-gw-d_bin_mean'],
                         mode='markers+lines',
                         name='Mean +/- STD',
                         line=dict(color='black'),
                         opacity=.55,
                         error_x_array=bin_df['point-gw-d_std'],
                         showlegend=False),
              row=4, col=1)

fig.add_trace(go.Scatter(x=residual_df['Time_Step'],
                         y=residual_df['point-gw-e'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='GW-E',
                         line=dict(color=color10_16[4]),
                         showlegend=True),
              row=5, col=1)
fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
                         y=bin_df['point-gw-e_bin_mean'],
                         mode='markers+lines',
                         name='Mean +/- STD',
                         line=dict(color='black'),
                         opacity=.55,
                         error_x_array=bin_df['point-gw-e_std'],
                         showlegend=True),
              row=5, col=1)

fig.update_layout(height=1000, width=500,
                  title_text="GW Damage Index Point Monitors")

fig.show()

In [71]:
color10_16 = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF',
              '#FECB52']
fig = make_subplots(
    rows=5,
    cols=1,
    shared_xaxes=False,
    x_title="Time Step",
    y_title="Pressure (Pa)")
# subplot_titles=("Point A", "Quadrant 2"))

fig.add_trace(go.Scatter(x=residual_df['Time_Step'],
                         y=residual_df['point-pressure-a'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='Pressure-A',
                         line=dict(color=color10_16[0])),
              row=1, col=1)
fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
                         y=bin_df['point-pressure-a_bin_mean'],
                         mode='markers+lines',
                         name='Mean +/- STD',
                         line=dict(color='black'),
                         opacity=.55,
                         error_y_array=bin_df['point-pressure-a_std'],
                         showlegend=False),
              row=1, col=1)

fig.add_trace(go.Scatter(x=residual_df['Time_Step'],
                         y=residual_df['point-pressure-b'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='Pressure-B',
                         line=dict(color=color10_16[1])),
              row=2, col=1)
fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
                         y=bin_df['point-pressure-b_bin_mean'],
                         mode='markers+lines',
                         name='Mean +/- STD',
                         line=dict(color='black'),
                         opacity=.55,
                         error_y_array=bin_df['point-pressure-b_std'],
                         showlegend=False),
              row=2, col=1)

fig.add_trace(go.Scatter(x=residual_df['Time_Step'],
                         y=residual_df['point-pressure-c'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='Pressure-C',
                         line=dict(color=color10_16[2]),
                         showlegend=True),
              row=3, col=1)
fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
                         y=bin_df['point-pressure-c_bin_mean'],
                         mode='markers+lines',
                         name='Mean +/- STD',
                         line=dict(color='black'),
                         opacity=.55,
                         error_y_array=bin_df['point-pressure-c_std'],
                         showlegend=False),
              row=3, col=1)

fig.add_trace(go.Scatter(x=residual_df['Time_Step'],
                         y=residual_df['point-pressure-d'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='Pressure-D',
                         line=dict(color=color10_16[3]),
                         showlegend=True),
              row=4, col=1)
fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
                         y=bin_df['point-pressure-d_bin_mean'],
                         mode='markers+lines',
                         name='Mean +/- STD',
                         line=dict(color='black'),
                         opacity=.55,
                         error_y_array=bin_df['point-pressure-d_std'],
                         showlegend=False),
              row=4, col=1)

fig.add_trace(go.Scatter(x=residual_df['Time_Step'],
                         y=residual_df['point-pressure-e'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='Pressure-E',
                         line=dict(color=color10_16[4]),
                         showlegend=True),
              row=5, col=1)
fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
                         y=bin_df['point-pressure-e_bin_mean'],
                         mode='markers+lines',
                         name='Mean +/- STD',
                         line=dict(color='black'),
                         opacity=.55,
                         error_y_array=bin_df['point-pressure-e_std'],
                         showlegend=True),
              row=5, col=1)

fig.update_layout(height=700, width=500,
                  title_text="Pressure Point Monitors")

fig.show()

In [72]:
color10_16 = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF',
              '#FECB52']
fig = make_subplots(
    rows=5,
    cols=1,
    shared_xaxes=True,
    x_title="Time Step",
    y_title="Velocity Magnitude (m/s)")
# subplot_titles=("Point A", "Quadrant 2"))

fig.add_trace(go.Scatter(x=residual_df['Time_Step'],
                         y=residual_df['point-vel-a'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='Velocity-A',
                         line=dict(color=color10_16[0])),
              row=1, col=1)

fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
                         y=bin_df['point-vel-a_bin_mean'],
                         mode='markers+lines',
                         name='Mean +/- STD',
                         line=dict(color='black'),
                         opacity=.55,
                         error_y_array=bin_df['point-vel-a_std'],
                         showlegend=False),
              row=1, col=1)


fig.add_trace(go.Scatter(x=residual_df['Time_Step'],
                         y=residual_df['point-vel-b'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='Velocity-B',
                         line=dict(color=color10_16[1])),
              row=2, col=1)
fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
                         y=bin_df['point-vel-b_bin_mean'],
                         mode='markers+lines',
                         name='Mean +/- STD',
                         line=dict(color='black'),
                         opacity=.55,
                         error_y_array=bin_df['point-vel-b_std'],
                         showlegend=False),
              row=2, col=1)

fig.add_trace(go.Scatter(x=residual_df['Time_Step'],
                         y=residual_df['point-vel-c'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='Velocity-C',
                         line=dict(color=color10_16[2]),
                         showlegend=True),
              row=3, col=1)
fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
                         y=bin_df['point-vel-c_bin_mean'],
                         mode='markers+lines',
                         name='Mean +/- STD',
                         line=dict(color='black'),
                         opacity=.55,
                         error_y_array=bin_df['point-vel-c_std'],
                         showlegend=False),
              row=3, col=1)

fig.add_trace(go.Scatter(x=residual_df['Time_Step'],
                         y=residual_df['point-vel-d'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='Velocity-D',
                         line=dict(color=color10_16[3]),
                         showlegend=True),
              row=4, col=1)
fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
                         y=bin_df['point-vel-d_bin_mean'],
                         mode='markers+lines',
                         name='Mean +/- STD',
                         line=dict(color='black'),
                         opacity=.55,
                         error_y_array=bin_df['point-vel-d_std'],
                         showlegend=False),
              row=4, col=1)

fig.add_trace(go.Scatter(x=residual_df['Time_Step'],
                         y=residual_df['point-vel-e'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='Velocity-E',
                         line=dict(color=color10_16[4]),
                         showlegend=True),
              row=5, col=1)
fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
                         y=bin_df['point-vel-e_bin_mean'],
                         mode='markers+lines',
                         name='Mean +/- STD',
                         line=dict(color='black'),
                         opacity=.55,
                         error_y_array=bin_df['point-vel-e_std'],
                         showlegend=True),
              row=5, col=1)

fig.update_layout(height=700, width=500,
                  title_text="Velocity Point Monitors")

fig.show()

In [73]:
color10_16 = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF',
              '#FECB52']
fig = make_subplots(
    rows=5,
    cols=1,
    shared_xaxes=True,
    x_title="Time Step",
    y_title="Shear Rate (1/s)")
# subplot_titles=("Point A", "Quadrant 2"))

fig.add_trace(go.Scatter(x=residual_df['Time_Step'],
                         y=residual_df['point-shear-a'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='Shear-A',
                         line=dict(color=color10_16[0])),
              row=1, col=1)
fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
                         y=bin_df['point-shear-a_bin_mean'],
                         mode='markers+lines',
                         name='Mean +/- STD',
                         line=dict(color='black'),
                         opacity=.55,
                         error_y_array=bin_df['point-shear-a_std'],
                         showlegend=False),
              row=1, col=1)

fig.add_trace(go.Scatter(x=residual_df['Time_Step'],
                         y=residual_df['point-shear-b'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='Shear-B',
                         line=dict(color=color10_16[1])),
              row=2, col=1)
fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
                         y=bin_df['point-shear-b_bin_mean'],
                         mode='markers+lines',
                         name='Mean +/- STD',
                         line=dict(color='black'),
                         opacity=.55,
                         error_y_array=bin_df['point-shear-b_std'],
                         showlegend=False),
              row=2, col=1)


fig.add_trace(go.Scatter(x=residual_df['Time_Step'],
                         y=residual_df['point-shear-c'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='Shear-C',
                         line=dict(color=color10_16[2]),
                         showlegend=True),
              row=3, col=1)
fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
                         y=bin_df['point-shear-c_bin_mean'],
                         mode='markers+lines',
                         name='Mean +/- STD',
                         line=dict(color='black'),
                         opacity=.55,
                         error_y_array=bin_df['point-shear-c_std'],
                         showlegend=False),
              row=3, col=1)

fig.add_trace(go.Scatter(x=residual_df['Time_Step'],
                         y=residual_df['point-shear-d'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='Shear-D',
                         line=dict(color=color10_16[3]),
                         showlegend=True),
              row=4, col=1)
fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
                         y=bin_df['point-shear-d_bin_mean'],
                         mode='markers+lines',
                         name='Mean +/- STD',
                         line=dict(color='black'),
                         opacity=.55,
                         error_y_array=bin_df['point-shear-d_std'],
                         showlegend=False),
              row=4, col=1)

fig.add_trace(go.Scatter(x=residual_df['Time_Step'],
                         y=residual_df['point-shear-e'],
                         mode='markers',
                         opacity=scatter_opacity,
                         name='Shear-E',
                         line=dict(color=color10_16[4]),
                         showlegend=True),
              row=5, col=1)
fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
                         y=bin_df['point-shear-e_bin_mean'],
                         mode='markers+lines',
                         name='Mean +/- STD',
                         line=dict(color='black'),
                         opacity=.55,
                         error_y_array=bin_df['point-shear-e_std'],
                         showlegend=True),
              row=5, col=1)

fig.update_layout(height=700, width=500,
                  title_text="Shear Rate Point Monitors")

fig.show()

In [74]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=residual_df['Time_Step'],
                         y=residual_df['p-in'],
                         mode='markers',
                         opacity=scatter_opacity,))
fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
                         y=bin_df['p-in_bin_mean'],
                         mode='markers+lines',
                         name='Mean +/- STD',
                         line=dict(color='black'),
                         opacity=.55,
                         error_y_array=bin_df['p-in_std'],
                         showlegend=True))
fig.update_layout(
    title="Area Weighted Average Pressure Inlet",
    xaxis_title="Iteration",
    yaxis_title="Pressure (Pa)",
    legend_title=None,height=300, width=700,)
fig.show()

In [75]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=residual_df['Time_Step'],
                         y=residual_df['eds-hemolysis'],
                         mode='markers',
                         opacity=scatter_opacity,))
fig.add_trace(go.Scatter(x=bin_df['bin_centres'],
                         y=bin_df['eds-hemolysis_bin_mean'],
                         mode='markers+lines',
                         name='Mean +/- STD',
                         line=dict(color='black'),
                         opacity=.55,
                         error_y_array=bin_df['eds-hemolysis_std'],
                         showlegend=True))
fig.update_layout(
    title="MWA Outlet EDS Hemolysis",
    xaxis_title="Iteration",
    yaxis_title="Damage Index",
    legend_title=None)
fig.show()


